In [ ]:
import numpy as np
import cv2
import math
import os

import random

from matplotlib import pyplot as plt
from collections import deque
from google.colab import drive
from google.colab.patches import cv2_imshow

from skimage import data, img_as_float
from skimage.metrics import structural_similarity as ssim

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Testbilder
house = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/Haus.JPEG', cv2.IMREAD_GRAYSCALE)
house_rgb = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/Haus.JPEG', cv2.IMREAD_COLOR)
pano = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/KoelnPano01.jpg', cv2.IMREAD_COLOR)
europa = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/europa.png', cv2.IMREAD_GRAYSCALE)
geheim = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/BV2_Geheim.jpg', cv2.IMREAD_GRAYSCALE)
maus = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/Testbilder/Maus_und_co.jpg', cv2.IMREAD_GRAYSCALE)
# cv2_imshow(geheim)

Aufgabe 01

In [ ]:
# Rauschen aus Abgabe 01
# Verbesserte Version meines Körnungsfilters (ChatGPT)
def grainFilter(image, grain):
  grainImage = np.zeros_like(image, dtype=float)
  resultImage = np.zeros_like(image, dtype=int)

  for y in range(image.shape[0]):
    for x in range(image.shape[1]):
      ranFloat = grain + random.random() * (1 - grain)
      grainImage[y][x] = np.array([ranFloat], dtype = float)
      resultImage[y][x] = np.array([int(image[y][x] * grainImage[y][x])], dtype = int)

  return resultImage

In [ ]:
def salt_and_pepper_noise(image, frequency):
  result_image = np.zeros_like(image)
  for y in range(image.shape[0]):
    for x in range(image.shape[1]):
      random_float = np.random.rand(1)
      if random_float < frequency:
        result_image[y][x] = 0
      elif random_float > (1-frequency):
        result_image[y][x] = 255
      else:
        result_image[y][x] = image[y][x]

  return result_image

In [ ]:
def blur_image(image, kernel):
  kernel = kernel if kernel % 2 == 1 else (kernel - 1)
  return cv2.GaussianBlur(image,(kernel,kernel),0)

In [ ]:
#BV1
def lin_contrast_spread(image, t0:int, t1:int):
  new_image = np.zeros_like(image)
  new_lut = [np.zeros(256),np.zeros(256)]# 8-bit
  k = 255
  for i in range(0, 256):
    if i <= t0:
      new_lut[1][i] = 0
    elif i > t1:
      new_lut[1][i] = k
    elif i > t0 and i <= t1:
      new_lut[1][i] = k/(t1-t0) * (i-t0)
  for y in range(0, image.shape[0]):# height
    for x in range(0, image.shape[1]):# width
      g_value = image[y, x]
      new_image[y, x] = new_lut[1][g_value]
  return new_image

In [ ]:
def reverse_gray(image):
  result_image = np.zeros_like(image)

  for y in range(image.shape[0]):
    for x in range(image.shape[1]):
      result_image[y][x] = 255 - image[y][x]

  return result_image

In [ ]:
def brightnessCorr(img, value):
  result_image = np.zeros_like(img)
  for y in range(img.shape[0]):
    for x in range(img.shape[1]):
      result_image[y][x] = np.round(np.clip(img[y][x] * value, 0, 255)).astype(int)

  return result_image

In [ ]:
# Filter
test_image = maus
#cv2_imshow(test_image)

# Rauschen aus Abgabe 01
image_grain = grainFilter(test_image, 0.55)

# Salt and Pepper Rauschen
image_sp = salt_and_pepper_noise(test_image, 0.015)

# Blur
image_blur = blur_image(test_image, 15)

# Kontrast
image_contrast = lin_contrast_spread(test_image, 80, 200)

# Helligkeitskorrektur
image_bright = brightnessCorr(test_image, 0.4)

In [ ]:
# SSIM
print(ssim(test_image, image_grain))
print(ssim(test_image, image_sp))
print(ssim(test_image, image_blur))
print(ssim(test_image, image_contrast))
print(ssim(test_image, image_bright))

0.5678692604484143
0.5545423956119784
0.5301450497596957
0.5446510624403339
0.5579997901778425


In [ ]:
#cv2_imshow(image_grain)
#cv2_imshow(image_sp)
#cv2_imshow(image_blur)
#cv2_imshow(image_contrast)
#cv2_imshow(image_bright)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/test_image.png', test_image)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/image_grain.png', image_grain)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/image_sp.png', image_sp)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/image_blur.png', image_blur)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/image_bright.png', image_bright)

Aufgabe 02

In [ ]:
# Nachricht verstecken
def hideMessage(image, message):
  resultImage = image
  for y in range(resultImage.shape[0]):
    for x in range(resultImage.shape[1]):
      if(message[y][x] > 100):#5
        resultImage[y][x][0] |= 1
      else:
        resultImage[y][x][0] &= ~1

  return resultImage

In [ ]:
# Nachricht herausholen
def findMessage(image):
  bitImages = []
  for bit in range(8):
    bitMask = 1 << bit
    bitPlane = cv2.bitwise_and(image[:, :, 0], bitMask)
    bitImages.append(bitPlane)

  for i, bitImage in enumerate(bitImages):
    cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/blue_bit_{i}.png', bitImage)

In [ ]:
def makeImageVisible(image):
  resultImage = np.zeros_like(image)
  for y in range(image.shape[0]):
    for x in range(image.shape[1]):
      resultImage[y][x] = 255 if (image[y][x] == 1) else 0
  return resultImage

In [ ]:
#Demo
newImage = hideMessage(house_rgb, geheim)
#findMessage(newImage)
#cv2_imshow(house_rgb)
#cv2_imshow(newImage)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/house_rgb.png', house_rgb)
cv2.imwrite('/content/drive/My Drive/Colab Notebooks/BV2/house_rgb_message.png', newImage)

True

In [ ]:
#bild0 = cv2.imread('/content/drive/My Drive/Colab Notebooks/BV2/blue_bit_0.png', cv2.IMREAD_GRAYSCALE)
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/blue.png', bild0)
#cv2_imshow(makeImageVisible(bild0))
#cv2.imwrite(f'/content/drive/My Drive/Colab Notebooks/BV2/blue_bit0_message.png', makeImageVisible(bild0))